In [11]:
load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [16]:
%%cython --annotate --compile-args=-fopenmp --link-args=-gomp

import numpy as np

import cython
from cython.parallel import prange
cimport openmp

cimport numpy as np
from libc.math cimport sin, cos, pi

#FLOAT64 = np.float64
ctypedef np.float64_t FLOAT64_t

@cython.boundscheck(False)
@cython.initializedcheck(False)  # turn off negative index wrapping for entire function
def initialize_trig_values(int dim,
                           np.ndarray zeta,
                           np.ndarray cos_zeta,
                           np.ndarray sin_zeta):
    
    cdef FLOAT64_t [:, :] z = zeta
    cdef FLOAT64_t [:, :] C = cos_zeta
    cdef FLOAT64_t [:, :] S = sin_zeta
    cdef int i, j
    
    with nogil:
        for i in prange(dim):
            for j in prange(i):
                C[i,j] = cos(z[i,j])
                S[i,j] = sin(z[i,j])
            

@cython.boundscheck(False)
@cython.initializedcheck(False)  # wraparound: turn off negative index wrapping for entire function
def HyperSphere_lower_triangular(np.ndarray L_arr,
                                 int dim,
                                 np.ndarray cos_zeta,
                                 np.ndarray sin_zeta):
    
    #cdef np.ndarray L_arr = np.zeros((dim, dim), dtype=FLOAT64)
    # Memoryviews of the numpy arrays
    cdef FLOAT64_t [:, :] L = L_arr
    cdef FLOAT64_t [:, :] C = cos_zeta
    cdef FLOAT64_t [:, :] S = sin_zeta

    cdef int r, s, j
    cdef FLOAT64_t L_rs
    
    with nogil:
        L[0,0] = 1.0
        for r in prange(dim):
            L_rs = 1.0
            for j in prange(r):
                L_rs *= S[r,j]
            L[r,r] = L_rs
            for s in prange(r):
                L_rs = C[r,s]
                for j in prange(s):
                    L_rs *= S[r,j]
                L[r,s] = L_rs
    #return L_arr
    

@cython.boundscheck(False)
@cython.initializedcheck(False)  # turn off negative index wrapping for entire function
def HyperSphere_lower_triangular_derivative(np.ndarray dL_arr,
                                            int dim,
                                            int dr,
                                            int ds,
                                            np.ndarray cos_zeta,
                                            np.ndarray sin_zeta):                                            
    # Memoryviews of the numpy arrays
    cdef FLOAT64_t [:, :] dL = dL_arr
    cdef FLOAT64_t [:, :] C = cos_zeta
    cdef FLOAT64_t [:, :] S = sin_zeta
    
    cdef FLOAT64_t dL_drs
    cdef int r, s, j
    
    with nogil:
        for s in prange(dr):
            if ds <= s:
                dL_drs = C[dr,s] if s != ds else -S[dr,s]
                for j in prange(s):
                    dL_drs *= S[dr,j] if j != ds else C[dr,j]
                dL[dr,s] = dL_drs
        # now set the diagonal, i.e. s = dr
        dL_drs = 1.0 
        for j in prange(dr):
            dL_drs *= S[dr,j] if j != ds else C[dr,j]
        dL[dr,dr] = dL_drs

    #return dL_arr



LinkError: command 'gcc' failed with exit status 1

In [13]:
from __future__ import print_function
from math import pi, sin, cos

import random
import timeit
import numpy
# Run the cell immediately above first
#import hypersphere_cython as hs

# =============================================================================
# Cythonize construction of matrices and gradient for efficiency
# =============================================================================
class HyperSphere(object):
    """Parameterizes the d-1-dimensional surface of a d-dimensional hypersphere
    using a lower triangular matrix with d*(d-1)/2 parameters, each in the 
    interval (0, pi).
    """
    def __init__(self, dim, zeta=[]):
        m = dim*(dim-1)//2
        self.dim = dim
        if isinstance(zeta, (list, tuple, np.ndarray)) and len(zeta):
            assert len(zeta) == m, "Expecting {0}*({0}-1)/2 elements".format(dim)
        elif isinstance(zeta, (int, float, np.float64, np.int64)):
            zeta = [zeta]
        else:
            zeta = [pi/4.0]*m
        zeta_lt = np.zeros((dim, dim))
        # lower triangular indices, offset -1 to get below-diagonal elements
        for z, ind in zip(zeta, zip(*np.tril_indices(dim, -1))):
            zeta_lt[ind] = z
        # set the diagonal to 1
        #np.fill_diagonal(zeta_lt.s, 1.0)

        self.zeta = zeta_lt
        
        # initialize trig values 
        self._cos_zeta = None
        self._sin_zeta = None
        self._initialize_trig_values()
        
        self._lt = None
        # CAUTION do not cache values at initialization without making sure
        # clone_with_theta will trigger their recalculation !
        #self._lt = self._lower_triangular()
       
    def _initialize_trig_values(self):
        cos_zeta = np.zeros((self.dim, self.dim), dtype=np.float64)
        sin_zeta = np.zeros((self.dim, self.dim), dtype=np.float64)
        
        initialize_trig_values(self.dim, self.zeta, cos_zeta, sin_zeta)
        
        self._cos_zeta = cos_zeta
        self._sin_zeta = sin_zeta
        
    @property
    def cos_zeta(self):
        return self._cos_zeta

    @property
    def sin_zeta(self):
        return self._sin_zeta

    # see HyperSphere_test for pure Python equivalent
    def _lower_triangular(self):
        if self._lt is None:
            lt = np.zeros((self.dim, self.dim))
            HyperSphere_lower_triangular(lt,
                                         self.dim,
                                         self.cos_zeta,
                                         self.sin_zeta)
            self._lt = lt
        return self._lt
    
    @property
    def correlation(self):
        lt = self._lower_triangular()
        corr = lt.dot(lt.T)
        # this is not strictly needed, 
        # but numerical error may cause some diagonal values to be
        # slightly off
        np.fill_diagonal(corr, 1.0)
        return corr

    # see HyperSphere_test for pure Python equivalent
    def _lower_triangular_derivative(self):
        dim = self.dim
        #zeta = self.zeta
        cos_zeta = self.cos_zeta
        sin_zeta = self.sin_zeta
        
        dLstack = []
        for dr, ds in zip(*np.tril_indices(dim, -1)):
            dL = np.zeros((dim, dim), dtype=np.float64)
            
            HyperSphere_lower_triangular_derivative(dL, dim, dr, ds,
                                                    cos_zeta, sin_zeta)
            dLstack.append(dL)
        return dLstack
    
    @property
    def gradient(self):
        L = self._lower_triangular()
        dLstack = self._lower_triangular_derivative()
        gradstack = []
        for dL in dLstack:
            dLLt = dL.dot(L.T)
            grad = dLLt + dLLt.T
            gradstack.append(grad)
        return gradstack


In [14]:
# =============================================================================
# Pure Python implememntation, compare to Cython version for testing
# =============================================================================
class HyperSphere_pure(HyperSphere):
    """Parameterizes the d-1-dimensional surface of a d-dimensional hypersphere
    using a lower triangular matrix with d*(d-1)/2 parameters, each in the 
    interval (0, pi).
    """
    def __init__(self, dim, zeta=[]):
        m = dim*(dim-1)//2
        self.dim = dim
        if isinstance(zeta, (list, tuple)) and len(zeta):
            assert len(zeta) == m, "Expecting {0}*({0}-1)/2 elements".format(dim)
        elif isinstance(zeta, (int, float, np.float64, np.int64)):
            zeta = [zeta]
        else:
            zeta = [pi/4.0]*m
        zeta_lt = np.zeros((dim, dim))
        # lower triangular indices, offset -1 to get below-diagonal elements
        for th, ind in zip(zeta, zip(*np.tril_indices(dim,-1))):
            zeta_lt[ind] = th
        # set the diagonal to 1
        for i in range(dim):
            zeta_lt[i,i] = 1.0
        self.zeta = zeta_lt
        self._lt = None
        #self._lt = self._lower_triangular()
            
    def _lower_triangular(self):
        if self._lt is None:
            dim = self.dim
            zeta = self.zeta
            L = np.zeros((dim, dim), dtype=np.float64)
            # auxilary array to avoid recomputing sin
            sin_r = np.zeros(dim, dtype=np.float64)
            for r in range(dim):
                L_rr = 1.0
                for j in range(r):
                    sin_rj = sin(zeta[r,j])
                    sin_r[j] = sin_rj
                    L_rr *= sin_rj
                L[r,r] = L_rr
                for s in range(r):
                    L_rs = cos(zeta[r,s])
                    for j in range(s):
                        L_rs *= sin_r[j]
                    L[r,s] = L_rs
            self._lt = L
        return self._lt
    
#    @property
#    def correlation(self):
#        lt = self._lower_triangular()
#        return lt.dot(lt.T)

    def _lower_triangular_derivative(self):
        dim = self.dim
        zeta = self.zeta
        #dL[0,0] = 0.0
        dLstack = []
        # auxiliary arrays to avoid recomputing trig functions
        C = np.zeros((dim, dim), dtype=np.float64)
        S = np.zeros((dim, dim), dtype=np.float64)
        for r, s in zip(*np.tril_indices(dim, -1)):
            zeta_rs = zeta[r,s]
            C[r,s] = cos(zeta_rs)
            S[r,s] = sin(zeta_rs)
        for dr, ds in zip(*np.tril_indices(dim, -1)):
            dL = np.zeros((dim, dim), dtype=np.float64)
            for s in range(dr):
                if ds <= s:
                    dL_drs = C[dr,s] if s != ds else -S[dr,s]
                    for j in range(s):
                        dL_drs *= S[dr,j] if j != ds else C[dr,j]
                    dL[dr,s] = dL_drs
            # now set the diagonal, i.e. s = dr
            dL_drs = 1.0
            for j in range(dr):
                dL_drs *= S[dr,j] if j != ds else C[dr,j]
            dL[dr,dr] = dL_drs
            dLstack.append(dL)
        return dLstack

In [15]:
def test_correlation(dim, pure=True):
    m = dim*(dim-1)//2
    zeta = np.array([random.uniform(0, pi) for i in range(m)])
    h = HyperSphere_pure(dim, zeta) if pure else HyperSphere(dim, zeta)
    return h.correlation
    
def test_gradient(dim, pure=True):
    m = dim*(dim-1)//2
    zeta = np.array([random.uniform(0, pi) for i in range(m)])
    h = HyperSphere_pure(dim, zeta) if pure else HyperSphere(dim, zeta)
    return h.gradient

In [10]:
%matplotlib

import pandas as pd
import seaborn as sns

from collections import defaultdict

template = "{:3} {:6} {:11} {:10.3f} sec, {:15.9f} usec per repetition"

hc = HyperSphere(6)
hp = HyperSphere_pure(6)
#ho = HyperSphere_pure_original(4)

print("\nChecking Correlation for equivalence to pure python")
diff = hc.correlation - hp.correlation
print(diff.min(), diff.max())
print("should be zero")
print("\nChecking Gradient for equivalence to pure python")
for gc, gp in zip(hc.gradient, hp.gradient):
    diff = gc - gp
    print(diff.min(), diff.max())
print("All should be zero")


Using matplotlib backend: MacOSX


NameError: name 'HyperSphere' is not defined

ERROR:tornado.application:Exception in callback functools.partial(<function wrap.<locals>.null_wrapper at 0x1125d08c8>)
Traceback (most recent call last):
  File "/Users/johnbauer/anaconda/envs/py3/lib/python3.6/site-packages/tornado/ioloop.py", line 605, in _run_callback
    ret = callback()
  File "/Users/johnbauer/anaconda/envs/py3/lib/python3.6/site-packages/tornado/stack_context.py", line 277, in null_wrapper
    return fn(*args, **kwargs)
  File "/Users/johnbauer/anaconda/envs/py3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 265, in enter_eventloop
    self.eventloop(self)
  File "/Users/johnbauer/anaconda/envs/py3/lib/python3.6/site-packages/ipykernel/eventloops.py", line 277, in loop_cocoa
    show.mainloop()
AttributeError: 'function' object has no attribute 'mainloop'


In [17]:
def test(function, dimension, pure, N):

    test = "test_{}({}, pure={})".format(function, dimension, pure)
    time = timeit.timeit(test,
                           setup="from __main__ import test_{}".format(function),
                           number=N)
    return dict(function=function, dimension=dimension,
                code="Python" if pure else "Cython",
                N=N,
                time=time)

class TimeitData():
    def __init__(self):
        self.data = defaultdict(list)

    def add_result(self, result_dict):
        data = self.data
        for k, v in result_dict.items():
            data[k].append(v)

    @property
    def dataframe(self):
        return pd.DataFrame(self.data)

td = TimeitData()

N = 200
max_dim = 31 # was 61; suppress test code when set to 2

usec = 1e6 / N

for d in range(2,max_dim):
    for f in ["correlation", "gradient"]:
        for pure in [False, True]:
            result_dict = test(f, d, pure, N)
            time = result_dict["time"]
            print(template.format(d,
                                  "Python" if pure else "Cython",
                                  f,
                                  time,
                                  time*usec))
            td.add_result(result_dict)

# read cached data if tests are suppressed
if max_dim == 2:
    df = pd.read_csv("HyperSphere_test_60.csv")
else:
    df = td.dataframe

df["color"] = df.code.replace(["Cython", "Python"], ["r", "b"])
df['parameters'] = df.dimension * (df.dimension - 1) / 2

df_corr = df[df.function == "correlation"]
df_grad = df[df.function == "gradient"]

df_corr.plot.scatter('dimension', 'time', c=df.color, title='Correlation')
df_grad.plot.scatter('dimension', 'time', c=df.color, title='Gradient')

df_corr.plot.scatter('parameters', 'time', c=df.color, title='Correlation')
df_grad.plot.scatter('parameters', 'time', c=df.color, title='Gradient')

sns.lmplot(x='dimension', y='time', data=df_corr, order=2, hue="code")
sns.lmplot(x='dimension', y='time', data=df_grad, order=2, hue="code")

sns.lmplot(x='parameters', y='time', data=df_corr, order=2, hue="code")
sns.lmplot(x='parameters', y='time', data=df_grad, order=2, hue="code")

df_corr_cython = df_corr[df_corr.code == 'Cython']
df_corr_python = df_corr[df_corr.code == 'Python']

df_corr_cython.index = range(len(df_corr_cython))
df_corr_python.index = range(len(df_corr_python))

df_corr_compare = pd.concat([df_corr_cython, df_corr_python], keys=['cython', 'python'], axis=1)
df_corr_compare['ratio'] = df_corr_compare[('python', 'time')] / df_corr_compare[('cython', 'time')]

df_grad_cython = df_grad[df_grad.code == 'Cython']
df_grad_python = df_grad[df_grad.code == 'Python']

df_grad_cython.index = range(len(df_grad_cython))
df_grad_python.index = range(len(df_grad_python))

df_grad_compare = pd.concat([df_grad_cython, df_grad_python], keys=['cython', 'python'], axis=1)
df_grad_compare['ratio'] = df_grad_compare[('python', 'time')] / df_grad_compare[('cython', 'time')]


df_corr_ratio = pd.DataFrame({ 'ratio' : df_corr_compare['ratio'], 'parameters' : df_corr_compare[('cython', 'parameters')]})
df_grad_ratio = pd.DataFrame({ 'ratio' : df_grad_compare['ratio'], 'parameters' : df_grad_compare[('cython', 'parameters')]})

sns.lmplot(x='parameters', y='ratio', data=df_corr_ratio, order=4)
sns.lmplot(x='parameters', y='ratio', data=df_grad_ratio, order=4)


  2 Cython correlation      0.021 sec,   104.001185000 usec per repetition
  2 Python correlation      0.014 sec,    70.361784997 usec per repetition
  2 Cython gradient         0.024 sec,   118.949954999 usec per repetition
  2 Python gradient         0.023 sec,   113.014780004 usec per repetition
  3 Cython correlation      0.013 sec,    63.479060000 usec per repetition
  3 Python correlation      0.021 sec,   102.945435001 usec per repetition
  3 Cython gradient         0.018 sec,    92.185090002 usec per repetition
  3 Python gradient         0.023 sec,   117.126099999 usec per repetition
  4 Cython correlation      0.009 sec,    43.212005003 usec per repetition
  4 Python correlation      0.012 sec,    57.746215002 usec per repetition
  4 Cython gradient         0.019 sec,    95.255850001 usec per repetition
  4 Python gradient         0.027 sec,   136.005515001 usec per repetition
  5 Cython correlation      0.010 sec,    49.305670000 usec per repetition
  5 Python correlation   

 29 Cython gradient         1.747 sec,  8732.639724999 usec per repetition
 29 Python gradient         6.223 sec, 31117.207699999 usec per repetition
 30 Cython correlation      0.084 sec,   419.845135002 usec per repetition
 30 Python correlation      0.301 sec,  1506.689470002 usec per repetition
 30 Cython gradient         1.785 sec,  8924.115455002 usec per repetition
 30 Python gradient         7.129 sec, 35643.889700000 usec per repetition
